# DB
### lets make a simple DB
- im gonna use sqlite since it's already included in python and we have a very simple use case
- i need a user_session table, which gathers all data relevant to how the user interacted with the recommendations
- this table will be consumed by:  
    1) monitoring model performance to track clicks and likes  
    2) build user_feedback "matrix" for ranker  
    3) ANN retraining pipeline - my idea is to rebuild/re-train the index once it's seen 100 (or so) new contracts  
- because of limitations of ANNOY library (from Spotify) i may need to have another table that maps contract_id to hashed embedding, although i think sqlite doesn't allow non-integer primary keys so i may have do use something else and be pretty hacky about it

In [47]:
import sqlite3
from datetime import datetime
import json
from typing import Dict

# not needed - just need toy data
import numpy as np
embedding  = [np.random.random() * 175 for _ in range(150)]


In [26]:
conn = sqlite3.connect(":memory:")
c = conn.cursor()
c.execute("""CREATE TABLE user_session (
            user_id integer,
            date string,
            user_contract_id integer,
            clicked1 integer,
            clicked2 integer,
            clicked3 integer,
            clicked4 integer,
            clicked5 integer,
            like1 integer,
            like2 integer,
            like3 integer,
            like4 integer,
            like5 integer,
            contract_id1 integer,
            contract_id2 integer,
            contract_id3 integer,
            contract_id4 integer,
            contract_id5 integer,
            embedding text)""")

In [56]:
sql_insert = """
    INSERT INTO user_session VALUES (
        :user_id, 
        :date, 
        :user_contract_id, 
        :clicked1, 
        :clicked2, 
        :clicked3, 
        :clicked4, 
        :clicked5, 
        :like1, 
        :like2, 
        :like3, 
        :like4, 
        :like5, 
        :contract_id1, 
        :contract_id2, 
        :contract_id3, 
        :contract_id4, 
        :contract_id5, 
        :embedding)
        """
sample = {
    "user_id" : 1,
    "date" : datetime.now().isoformat(),
    "user_contract_id" : -1,
    "clicked1" : 1,
    "clicked2" :1,
    "clicked3": 0,
    "clicked4" :0,
    "clicked5" :0,
    "like1" :-1,
    "like2" :1,
    "like3"  :0,
    "like4" : 0,
    "like5" : 0,
    "contract_id1" : 56,
    "contract_id2" : 765,
    "contract_id3" : 222,
    "contract_id4" :  789,
    "contract_id5" : 121,
    "embedding" : json.dumps(embedding)
}

In [84]:
import os
class UserSession:
    def __init__(self, table_path):
        self.conn = sqlite3.connect(table_path)
        self.c = self.conn.cursor()
        try:
            self.N = self.search_db("SELECT COUNT(*) FROM user_session")[0][0]
        except:
            self.create_table()

    def create_table(self):
        self.c.execute("""CREATE TABLE user_session (
            user_id integer,
            date string,
            user_contract_id integer,
            clicked1 integer,
            clicked2 integer,
            clicked3 integer,
            clicked4 integer,
            clicked5 integer,
            like1 integer,
            like2 integer,
            like3 integer,
            like4 integer,
            like5 integer,
            contract_id1 integer,
            contract_id2 integer,
            contract_id3 integer,
            contract_id4 integer,
            contract_id5 integer,
            embedding text)""")
        
    def insert_db(self, user_session_dict: Dict):
        with self.conn:
            self.c.execute(sql_insert, sample)
            
    def search_db(self, sql_query: str):
        with self.conn:
            self.c.execute(sql_query)
            return self.c.fetchall()
            
    def close_connection(self):
        self.conn.close()

In [85]:
US = UserSession("user_session.db")

In [87]:
US.insert_db(sample)

In [88]:
US.search_db("SELECT COUNT(*) FROM user_session")[0][0]

2

In [86]:
US.N

1

### let's try building hash_emb lookup

In [89]:
c.execute("""CREATE TABLE embed_hash_lookup (hash_emb string PRIMARY KEY, contract_id integer)""")

In [104]:
with conn:
    c.execute("SELECT COUNT(*) FROM embed_hash_lookup")
    print(c.fetchall())

[(40001,)]


In [98]:
def insert_here(user_session_dict):
    with conn:
        c.execute("INSERT INTO embed_hash_lookup VALUES (:hash_emb, :contract_id)", user_session_dict)

In [22]:
import hashlib
import numpy as np

def array_to_hashlib(arr):
    emb_str = "".join([str(_fl) for _fl in arr])
    return hashlib.sha256(bytes(emb_str, encoding='utf-8')).hexdigest()
def generator():
    embedding  = [np.random.random() * 175 for _ in range(150)]
    return {
        "hash_emb": array_to_hashlib(embedding),
        "contract_id": np.random.randint(-1, 123456)
    }

In [121]:
%%time
with conn:
    c.execute("SELECT contract_id FROM embed_hash_lookup WHERE hash_emb='1e4923ac24237c582d9ccac07bcd9c6701dc98e9d391eb564fb7de5f1b2f176f1'")
    res = c.fetchall()
    print(res)

[]
CPU times: user 200 µs, sys: 124 µs, total: 324 µs
Wall time: 241 µs


In [1]:
from db import EmbedHashLookup
EHL = EmbedHashLookup("embhash.db")

In [ ]:
%%time
for _ in range(1000000):
    EHL.insert_db(generator())

In [2]:
EHL.search_db("SELECT COUNT(*) FROM embed_hash_lookup")[0][0]

321173

### lets compare latency for primary key vs not

In [3]:
%%time
EHL.search_db("SELECT hash_emb FROM embed_hash_lookup WHERE contract_id=22")

CPU times: user 13.5 ms, sys: 6.83 ms, total: 20.3 ms
Wall time: 19.2 ms


[('84ed4281c69155c865162fec04ac91abb38d411a005cd1c4bfda38d31a7b4434',)]

In [4]:
%%time
EHL.search_db("SELECT contract_id FROM embed_hash_lookup WHERE hash_emb='84ed4281c69155c865162fec04ac91abb38d411a005cd1c4bfda38d31a7b4434'")

CPU times: user 432 µs, sys: 346 µs, total: 778 µs
Wall time: 460 µs


[(22,)]

In [5]:
%%time
EHL.find_contract_id('84ed4281c69155c865162fec04ac91abb38d411a005cd1c4bfda38d31a7b4434')

CPU times: user 263 µs, sys: 194 µs, total: 457 µs
Wall time: 300 µs


22

- huge difference

# consolidate

In [105]:
import sqlite3
from typing import Dict

class SmartContractDB:
    def __init__(self, table_path):
        self.conn = sqlite3.connect(table_path)
        self.c = self.conn.cursor()
        self.table_name = "smart_contracts"
        exists = self.search_db("SELECT name FROM sqlite_master WHERE type='table' AND name='{}';".format(self.table_name))
        if len(exists) == 0:
            self.create_table()

    
    def create_table(self):
        self.c.execute("""CREATE TABLE {} (
            hash_emb string PRIMARY KEY, 
            contract_id integer, 
            contract_name text,
            contract_history text,
            embedding text
            )""".format(self.table_name))
        
    def insert_db(self, hashemb_contract_id: Dict):
        with self.conn:
            self.c.execute("""INSERT INTO smart_contracts VALUES (
                :hash_emb, 
                :contract_id,
                :contract_name,
                :contract_history,
                :embedding
                )""", hashemb_contract_id)
            
    def find_contract_id(self, hash_emb) -> int:
        with self.conn:
            self.c.execute("SELECT contract_id FROM smart_contracts WHERE hash_emb=:hash_emb", {"hash_emb": hash_emb})
            result = self.c.fetchall()
            try:
                return result[0][0]
            except IndexError:
                return -1
            
        
    def find_contract_history(self, hash_emb) -> Dict:
        with self.conn:
            self.c.execute("SELECT contract_history FROM smart_contracts WHERE hash_emb=:hash_emb", {"hash_emb": hash_emb})
            result = self.c.fetchall()
            try:
                return {int(k): v for k, v in json.loads(result[0][0]).items()}
            except IndexError:
                empty_history = dict()
                return empty_history
            
    def find_contract_name(self, contract_id):
        with self.conn:
            self.c.execute("SELECT contract_name FROM smart_contracts WHERE contract_id=:contract_id", {"contract_id": contract_id})
            result = self.c.fetchall()
            try:
                return result[0][0]
            except IndexError:
                return ""

    def append_history(self, hash_emb: str, new_contract_info: Dict):
        """
        if embedding is in the DB, it updates the contract history
        to-do: if it is not present, it creates a new entry for it and updates contract history

        """
        embedding_history = self.find_contract_history(hash_present)
        embedding_history.update(new_contract_info)
        jsonify_history = json.dumps(embedding_history)
        with self.conn:
            self.c.execute("UPDATE smart_contracts SET contract_history=:contract_history WHERE hash_emb=:hash_emb",
                          {"hash_emb": hash_emb, "contract_history": jsonify_history})
        
    def search_db(self, sql_query: str):
        with self.conn:
            self.c.execute(sql_query)
            return self.c.fetchall()
    
    def close_connection(self):
        self.conn.close()

In [122]:
SCDB = SmartContractDB("smart_contracts.db")

In [125]:
SCDB.search_db("PRAGMA table_info(smart_contracts);")

[(0, 'hash_emb', 'string', 0, None, 1),
 (1, 'contract_id', 'INTEGER', 0, None, 0),
 (2, 'contract_name', 'TEXT', 0, None, 0),
 (3, 'contract_history', 'TEXT', 0, None, 0),
 (4, 'embedding', 'TEXT', 0, None, 0)]

In [111]:
SCDB.search_db("SELECT COUNT(*) FROM smart_contracts")[0][0]

0

In [112]:
import json
def generator():
    embedding  = [np.random.random() * 175 for _ in range(150)]
    gen = {
        "hash_emb": array_to_hashlib(embedding),
        "contract_id": np.random.randint(-1, 123456),
        "contract_history": {
            int(_synth_id) : np.random.randint(-2, 3)
            for _synth_id in np.random.randint(-1, 123456, size=np.random.randint(1,5))
        },
        "contract_name": "cool{}.sol".format(np.random.randint(1,100)),
        "embedding": None
    }
    gen["contract_history"] = json.dumps(gen["contract_history"])
    return gen

generator()

{'hash_emb': '0487d3abc523dfd2b6dfbac68fa4b525dd965e364fdaa0ebf5c854957af631a8',
 'contract_id': 51558,
 'contract_history': '{"110927": 0, "97468": 1}',
 'contract_name': 'cool82.sol',
 'embedding': None}

### populate

In [121]:
%%time
for _ in range(30000):
    SCDB.insert_db(generator())

CPU times: user 8.77 s, sys: 13.1 s, total: 21.8 s
Wall time: 24.3 s


### case 1 - embedding is in DB

In [80]:
hash_present = "218c9c32f7fdf9c6741181910a31e582d76adee59cb2d99e92a0b91c7e642d00"

contract_id = SCDB.find_contract_id(hash_present)
embedding_history = SCDB.find_contract_history(hash_present)
print(contract_id, embedding_history)

92236 {80147: -2, 65170: -1, 83381: 0, 90796: -2}


### case 2 - embedding not in DB

In [82]:
hash_not_present = "218c9c32f7sfdf9c6741181910a31e582d76adee59cb2d99e92a0b91c7e642d00"

contract_id = SCDB.find_contract_id(hash_not_present)
embedding_history = SCDB.find_contract_history(hash_not_present)
print(contract_id, embedding_history)

-1 {}


In [83]:
%%timeit
embedding_history = SCDB.find_contract_history(hash_not_present)


13.1 µs ± 159 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [84]:
%%timeit
embedding_history = SCDB.find_contract_history(hash_present)


19.8 µs ± 192 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [85]:
! du -ah | grep smart_contracts

640K	./smart_contracts.db


### how do we add to the history
### case 1 - embedding in DB

In [103]:
print(SCDB.find_contract_history(hash_present))
SCDB.append_history(hash_present, {77: -1})
SCDB.find_contract_history(hash_present)

{80147: -2, 65170: -1, 83381: 0, 90796: -2, 77: -1}


{80147: -2, 65170: -1, 83381: 0, 90796: -2, 77: -1}

### case 2 - embedding not in DB

In [95]:
print(SCDB.find_contract_history(hash_not_present))
SCDB.append_history(hash_not_present, {77: -1})
SCDB.find_contract_history(hash_not_present)

{}


{}

In [96]:
SCDB.find_contract_id(hash_not_present)

-1